In [1]:
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

# The dataset

This TensorFlow dataset is made up of 28 x 28 grayscale images into 10 classes (from 0 to 9) with pixel values between 0 and 255.

*Fashion MNIST datset* is made up of 60_000 training images and 10_000 test images.

This dataset has all the images in the same shape, so we have little preprocessing to do.